In [1]:
from dataclasses import dataclass
from pathlib import Path

data_file = Path("../Data/day6.txt").read_text()

EXAMPLE = """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""


@dataclass
class Cell:
    row: int
    column: int
    value: str

    @property
    def hash_value(self):
        return str((self.row, self.column))


class GuardMap:
    items: list[list[Cell]]
    traveled: set[Cell] = set()

    def __init__(self, items: list[list[Cell]]) -> None:
        self.items = items

    def print_map(self):
        for row in self.items:
            print("".join(map(lambda x: x.value, row)))

    def move_forward(self, start: Cell, direction: str):
        if direction == "^":
            return self.move_up(start)
        if direction == ">":
            return self.move_right(start)
        if direction == "v":
            return self.move_down(start)
        if direction == "<":
            return self.move_left(start)

        raise Exception("Not right!")

    def move_up(self, start: Cell):
        last_cell: Cell | None = None
        hit_wall = True
        for row in reversed(range(0, start.row)):
            cell = self.items[row][start.column]
            if cell.value == "#":
                hit_wall = False
                break

            self.items[row][start.column].value = "X"
            self.traveled.add(cell.hash_value)
            last_cell = cell

        return self.end_moving(last_cell, hit_wall)

    def move_down(self, start: Cell):
        last_cell: Cell | None = None
        hit_wall = True
        for row in range(start.row, len(self.items)):
            cell = self.items[row][start.column]
            if cell.value == "#":
                hit_wall = False
                break

            self.items[row][start.column].value = "X"
            self.traveled.add(cell.hash_value)
            last_cell = cell

        return self.end_moving(last_cell, hit_wall)

    def move_right(self, start: Cell):
        last_cell: Cell | None = None
        hit_wall = True
        for column in range(start.column, len(self.items[start.row])):
            cell = self.items[start.row][column]
            if cell.value == "#":
                hit_wall = False
                break

            self.items[start.row][column].value = "X"
            self.traveled.add(cell.hash_value)
            last_cell = cell

        return self.end_moving(last_cell, hit_wall)

    def move_left(self, start: Cell):
        last_cell: Cell | None = None
        hit_wall = True
        for column in reversed(range(0, start.column)):
            cell = self.items[start.row][column]
            if cell.value == "#":
                hit_wall = False
                break

            self.items[start.row][column].value = "X"
            self.traveled.add(cell.hash_value)
            last_cell = cell

        return self.end_moving(last_cell, hit_wall)

    def end_moving(self, last_cell: Cell, hit_wall: bool):
        assert last_cell is not None

        return self.items[last_cell.row][last_cell.column], hit_wall

    def find_character(self, search: str):
        for row_line in self.items:
            for cell in row_line:
                if cell.value == search:
                    return cell

        raise Exception("Nothing to be found here!")


def split_in_to_characters(string: str):
    return list(string)


def prepare(input: str):
    def map_row_line(args: tuple[int, list[str]]):
        row, row_line = args
        cells: list[Cell] = []
        for index, value in enumerate(row_line):
            cells.append(Cell(row=row, column=index, value=value))

        return cells

    guard_map = GuardMap(
        list(
            map(
                map_row_line, enumerate(map(split_in_to_characters, input.splitlines()))
            )
        )
    )

    return guard_map


data = prepare(data_file)
example_data = prepare(EXAMPLE)

In [2]:
def turn_right(direction: str):
    turns = {"^": ">", ">": "v", "v": "<", "<": "^"}

    return turns[direction]


def part1(guard_map: GuardMap):
    point = guard_map.find_character("^")
    assert point.value == "^"

    direction = point.value

    hit_wall = False
    while not hit_wall:
        point, hit_wall = guard_map.move_forward(point, direction)
        direction = turn_right(direction)

        if hit_wall:
            break

    guard_map.print_map()

    return len(guard_map.traveled)


example_result = part1(example_data)

print("example result is", example_result)

assert example_result == 41

result = part1(data)

print("result is", result)

assert result < 5126
assert result < 5125
assert result > 5003
assert result != 5004

....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#XXXXXXX.
.XXXXXXX#.
#XXXXXXX..
......#X..
example result is 41
........#.............................#......#................#..................#..................#.........#.#.#...............
...........................................#..#......................#............................#...........................#.#.
.............#..........#........#.#...........#........................#............................##...........................
............#.............#.....................................#........................#.....#..................................
.........#...........#................#................................#......#.....................#.............#..............#
...........................##.........XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX#.........
...........#.......#.......XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

AssertionError: 